<a href="https://colab.research.google.com/github/AnesCenanovic/Vje-ta-kaInteligencija/blob/main/VILab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyswarms


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.5 MB/s eta 0:00:00


In [3]:
from sklearn.datasets import load_iris
import numpy as np
import pyswarms as ps
data = load_iris()

X = data.data
y = data.target

y_new = np.zeros([X.shape[0], 3])

for i in range(X.shape[0]):
  y_new[i, y[i]] = 1.

y = y_new

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

def probs_function(p, X):
  W1 = p[0:80].reshape((n_inputs,n_hidden))
  b1 = p[80:100].reshape((n_hidden,))
  W2 = p[100:160].reshape((n_hidden,n_classes))
  b2 = p[160:163].reshape((n_classes,))

  z1 = X.dot(W1) + b1
  a1 = np.tanh(z1)
  logits = a1.dot(W2) + b2

  exp_scores = np.exp(logits)
  probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)


  return probs

from keras import losses

def forward_prop(params):

  probs = probs_function(params, X_train)

  cce = losses.CategoricalCrossentropy()
  loss = cce(y_train, probs).numpy()

  return loss

def f(x):
  n_particles = x.shape[0]
  values = [forward_prop(x[i]) for i in range(n_particles)]
  return np.array(values)


options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=
  options)

cost, pos = optimizer.optimize(f, iters=100)

def predict(pos) :
  probs = probs_function(pos,    X_test)
  return probs

(np.round(predict(pos)).astype('int') == y_test).mean()



2025-06-03 20:33:02,536 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.079
2025-06-03 20:33:18,984 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0789705440402031, best pos: [-1.24128542e+00  2.27993583e-01 -3.15212598e-01 -7.87411345e-02
  2.58464879e-02 -4.88470816e-01  6.99461770e-01 -4.27493262e+00
  1.67295742e-01  7.96978812e-02 -6.34391018e-01 -9.50955275e-01
  8.55990385e-01 -1.01391416e-03  9.07894960e-01  1.88178696e-01
  4.26000579e-01 -4.74669909e-01 -6.54519908e-01  4.56843244e-01
  1.73710072e+00 -5.82591433e-01  1.38117833e+00  3.36462628e-02
  1.40154392e+00  1.78135620e-01 -3.39732560e-02 -2.53282963e+00
  4.70792533e-02 -1.04691178e+00  8.84923988e-01  2.82128445e-01
 -7.17686658e-01 -1.28200828e+00 -5.13396406e-01 -3.34682350e-01
  9.72889447e-02 -4.31977205e-03 -2.86554559e-01 -3.38324645e-02
 -2.70548652e-01  8.76555383

np.float64(0.9703703703703703)